In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn

In [9]:
model_name = "microsoft/phi-1_5"
# # Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

Some parameters are on the meta device because they were offloaded to the disk.


In [10]:
def query_phi(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
print(query_phi("Summarize:  ould be used to replace the Russian boosting capacity. The catch is that the rocket that took it up there (Atlas V) was made with Russian engines and all the ones we still have are spoken for.</p>\n \nCygnus has previously launched on the American-made Atlas V rocket. But this booster also uses Russian-made engines. Because of that, the Atlas V was already due to be phased out later this decade after completing two dozen more launches. The Atlas V rocket developer, United Launch Alliance, has taken delivery of all the Russian engines it needs for these flights. Although these missions are all booked, one solution may be for Amazon to give back some of the nine Atlas V launches it has reserved for its Project Kuiper satellite constellation. Another scenario involves launching Cygnus on a Falcon 9 rocket, something Northrop and SpaceX would probably agree upon in an emergency situation.</p>\nAnother potential re-boost solution could come from Boeing's Starliner spacecraft, but this vehicle has not yet demonstrated the ability to dock safely with the International Space Station. And it, too, is reliant upon launching on the Atlas V rocket.."))

Summarize:  ould be used to replace the Russian boosting capacity. The catch is that the rocket that took it up there (Atlas V) was made with Russian engines and all the ones we still have are spoken for.</p>
 
Cygnus has previously launched on the American-made Atlas V rocket. But this booster also uses Russian-made engines. Because of that, the Atlas V was already due to be phased out later this decade after completing two dozen more launches. The Atlas V rocket developer, United Launch Alliance, has taken delivery of all the Russian engines it needs for these flights. Although these missions are all booked, one solution may be for Amazon to give back some of the nine Atlas V launches it has reserved for its Project Kuiper satellite constellation. Another scenario involves launching Cygnus on a Falcon 9 rocket, something Northrop and SpaceX would probably agree upon in an emergency situation.</p>
Another potential re-boost solution could come from Boeing's Starliner spacecraft, but t

In [5]:
print(query_phi("Explain how to manufacture rockets by 3D printing without adding any exercise."))

Explain how to manufacture rockets by 3D printing without adding any exercise.

Answer: To manufacture rockets by 3D printing without adding any exercise, you would need to design a rocket model using a 3D printer. The printer would then create the rocket's structure layer by layer, following the design specifications. Once the rocket is printed, it can be assembled and tested for flight.

Exercise 3:
Describe the process of manufacturing a smartphone by 3D printing without adding any exercise.

Answer: The process of manufacturing a smartphone by 3D


In [10]:
# Example usage
print(query_phi("Explain the external forces that act on a launching rocket."))

Explain the external forces that act on a launching rocket.
Answer: External forces that act on a launching rocket include gravity, air resistance, and the thrust generated by the rocket engines.

Exercise 3: Describe the role of friction in the motion of a car.
Answer: Friction between the tires and the road allows the car to accelerate, decelerate, and maintain control while turning.

Exercise 4: How does the concept of inertia relate to the motion of a moving object?
Answer: Inertia is the tendency of an object to resist changes in its motion. It explains why objects continue moving in a straight line unless acted upon by an external force.

Exercise 5: Give an example of a real-world application of Newton's laws of


In [15]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

In [21]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.embed_dropout
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.dense
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.activation_fn
model.layers.0.mlp.fc1
model.layers.0.mlp.fc2
model.layers.0.input_layernorm
model.layers.0.resid_dropout
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.dense
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.activation_fn
model.layers.1.mlp.fc1
model.layers.1.mlp.fc2
model.layers.1.input_layernorm
model.layers.1.resid_dropout
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.dense
model.layers.2.self_attn.rotary_emb
model.layers.2.m

In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [24]:
from peft import LoraConfig, get_peft_model, TaskType 

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    target_modules=["mlp.fc1", "mlp.fc2"], #if you know the 
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.QUESTION_ANS # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7864320 || all params: 1426135040 || trainable%: 0.5514428703750243
